In [31]:
from nltk.translate.bleu_score import corpus_bleu
import argparse
import json
import logging
import nltk
import os
import pandas as pd
from evaluation_utils import failed_generation_index, eval_dataset, get_nested_values, load_dataset, safe_loc, compute_precision, compute_recall, corpus_meteor

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
arguments = {
    "dataset": "../outputs/batch_run/test2/execution/Mistral-7B-Instruct-v0.2_rv16-bs2-p0_t0.2-topp0.95_executed.parquet.gzip",
    "gold": "../datasets/queries_with_execution_results_with_limit.parquet.gzip",
    "model": "Mistral-7B-Instruct-v0.2",
    "output": ".",
    "save_name": "test",
    "log_level": "warning",
    "log_file": "",
}

In [3]:
args = argparse.Namespace()
args.__dict__.update(arguments)
print(args)

numeric_log_level = getattr(logging, args.log_level.upper(), None)
if not isinstance(numeric_log_level, int):
    raise ValueError(f"Invalid log level: {args.log_level}.")
logging.basicConfig(filename=args.log_file if args.log_file else None, level=numeric_log_level)

if not os.path.exists(args.dataset):
    raise FileNotFoundError(f"The dataset file not found with path: {args.dataset}")

if args.gold != None and not os.path.exists(args.gold):
    raise FileNotFoundError(f"The gold dataset file not found with path: {args.gold}")

if args.preprocess_gold != None and not os.path.exists(args.preprocess_gold):
    raise FileNotFoundError(f"The preprocess gold dataset file not found with path: {args.preprocess_gold}")

nltk.download('wordnet', quiet=True)

Namespace(dataset='../outputs/batch_run/test2/execution/Mistral-7B-Instruct-v0.2_rv16-bs2-p0_t0.2-topp0.95_executed.parquet.gzip', gold='../datasets/queries_with_execution_results_with_limit.parquet.gzip', log_file='', log_level='warning', model='Mistral-7B-Instruct-v0.2', output='.', save_name='test')


In [4]:
df = load_dataset(args.dataset)
df_no_gen_fail = df.drop(failed_generation_index(df))
df_exec_timeout = df_no_gen_fail.loc[df_no_gen_fail['execution'] == 'timeout']
df_exec_fail = df_no_gen_fail.loc[df_no_gen_fail['execution'].str.startswith('exception')]
df_exec_empty = df_no_gen_fail.loc[df_no_gen_fail['execution'].isnull()]
df_exec_to_eval = df_no_gen_fail.drop(df_exec_timeout.index).drop(df_exec_fail.index).drop(df_exec_empty.index)
df_eval = eval_dataset(df_exec_to_eval)

In [5]:
df_gold = None
df_gold_exec_timeout = None
df_gold_exec_fail = None
df_gold_exec_empty = None
df_gold_exec_to_eval = None
df_gold_eval = None
if args.gold != None:
    df_gold = load_dataset(args.gold)
    df_gold_exec_timeout = df_gold.loc[df_gold['execution'] == 'timeout']
    df_gold_exec_fail = df_gold.loc[df_gold['execution'].str.startswith('exception')]
    df_gold_exec_empty = df_gold.loc[df_gold['execution'].isnull()]
    df_gold_exec_to_eval = df_gold.drop(df_gold_exec_timeout.index).drop(df_gold_exec_fail.index).drop(df_gold_exec_empty.index)
    df_gold_eval = eval_dataset(df_gold_exec_to_eval, "gold_eval")
else:
    with open(args.preprocess_gold, "r") as f:
        data = json.load(f)
    df_gold = pd.read_json(data['df_gold'])
    df_gold_exec_timeout = pd.read_json(data['df_gold_exec_timeout'])
    df_gold_exec_fail = pd.read_json(data['df_gold_exec_fail'])
    df_gold_exec_empty = pd.read_json(data['df_gold_exec_empty'])
    df_gold_exec_to_eval = pd.read_json(data['df_gold_exec_to_eval'])
    df_gold_eval = pd.read_json(data['df_gold_eval'])

In [6]:
df_merged_eval = df_eval.copy()
df_merged_eval["gold_eval"] = df_merged_eval.apply(lambda x: safe_loc(x, df_gold_eval, "gold_eval", default=None), axis=1)
df_merged_eval["precision"] = df_merged_eval.apply(lambda x: compute_precision(get_nested_values(x['eval']), get_nested_values(x['gold_eval'])), axis=1)
df_merged_eval["recall"] = df_merged_eval.apply(lambda x: compute_recall(get_nested_values(x['eval']), get_nested_values(x['gold_eval'])), axis=1)

In [ ]:
m_precision = df_merged_eval['precision'].mean()
m_recall = df_merged_eval['recall'].mean()
m_fscore = 2*m_precision*m_recall/(m_precision+m_recall)

bleu_score = corpus_bleu([[x.split()] for x in df_no_gen_fail['target']], [x.split() for x in df_no_gen_fail['translated_prompt']])
meteor_score = corpus_meteor(df_no_gen_fail['target'], df_no_gen_fail['translated_prompt'])


In [ ]:
serie = pd.Series(data=
    {
        "model_name": args.model,
        "num_rows": len(df),
        "num_gen_fail": len(df.loc[df['has_error'] == True]),
        "num_exec_timeout": len(df_exec_timeout),
        "num_exec_fail": len(df_exec_fail),
        "num_exec_empty": len(df_exec_empty),
        "num_exec_to_eval": len(df_exec_to_eval),
        "num_eval": len(df_eval),
        "num_eval_empty": len(df_eval.loc[df_eval['eval'].map(len) == 0]),
        "bleu_score": bleu_score,
        "meteor_score": meteor_score,
        "precision": m_precision,
        "recall": m_recall,
        "f1score": m_fscore
    })